In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

os.chdir("/home/tugn232/projects/shap-e/")

In [2]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [8]:
batch_size = 4 # the number of 3D objects to generate
guidance_scale = 10.0
prompt = "a cylindrical mug"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [9]:
#from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
render_mode = 'nerf' # you can change this to 'stf'
size = 128 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images, prompt+'_'+str(i)+'.gif'))

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAOTRq+DPquDOqt/MqNvLqNvKqNrJptrIpdfIptjGpdbFpNXFpd…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAOjq6ufo6OXn5+Tm5uPk5eHk5OLj4+Hi4+Di4uDh4t/h4uDg4N…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAPbm7O/c4+vV3OfS2efO1eXO1eTN1eLN1OHL0uLJ0d/J0OHGz9…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAABcUDBYUDBUUDBUUCxUTDBUTCxQTDBQTCxQSDBQSCxMTCxMSDB…

In [21]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)